In [1]:
import numpy as np
import pandas as pd
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import json
from pyproj import Transformer
import math
import shapely
from time import sleep
import datetime
import panel as pn

%matplotlib tk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.path as mpltPath
import matplotlib.patches as patches
import os

from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy import ndimage as ndi
from skimage import feature

import pytz
from tzwhere import tzwhere
from pysolar.solar import get_azimuth, get_altitude, radiation

#!pip install "laspy[lazrs,laszip]" #ensure laz handler installed
import laspy 
from treeSolarTools import *

In [8]:
#show the national Lidar map for user selection

lon,lat = 0,0
fig = plt.figure(figsize=(15,15), dpi=150, constrained_layout=True)
ax1 = fig.add_subplot(111)

df = pd.read_csv('2015StreetTreesCensus_TREES.csv')

def onclick(event):
    global lon
    global lat
    lon, lat = event.xdata, event.ydata
    print(lat, lon)

cid = fig.canvas.mpl_connect('button_press_event', onclick)
#print(cid)
features = readGeoJSON('lidarBoundaries.geojson')
for feature in features:
    points,name = footprintPointsFromGeoJSON(feature)
    path = mpltPath.Path(points)
    patch = patches.PathPatch(path, facecolor='whitesmoke', lw=0.3)
    ax1.add_patch(patch)
    
#plt.scatter(df['x_sp'],df['y_sp'],s=0.05,c='green')

ratio = 1.2
xMin, xMax = ax1.get_xlim()
yMin, yMax = ax1.get_ylim()
ax1.set_aspect( abs( ( xMax - xMin ) / ( yMin - yMax ) ) * ratio )
ax1.set_xlim(-132, -62)   
ax1.set_ylim(22, 52)

plt.show()

43.947858145336795 -78.14173597323366
40.6771583719837 -73.88925769008843


In [9]:
#lat,lon = 42.317398154639406, -83.08877069697685 # Detroit test
#lat,lon = 42.44963137138453, -76.47201539955707 # Cornell Botanic Gardens
#lat,lon = 37.126779357226944, -122.12067957145447 #Boulder Creek, CA

lidarArea = None
features = readGeoJSON('lidarBoundaries.geojson')
for feature in features:
    points,name = footprintPointsFromGeoJSON(feature)
    poly = shapely.geometry.Polygon(points)
    point = shapely.geometry.Point(lon,lat)
    if poly.intersects(point):
        lidarArea = '{}/'.format(name)
if lidarArea == None:
    print('Lidar not found for this area, please enter another lat,lon.')
else:
    print('Lidar found for: ',lidarArea)
    
#refine location selection with tree census map for NYC

if lidarArea == 'NY_NewYorkCity/':
    ix,iy = 0,0
    fig = plt.figure(figsize=(15,15), dpi=150, constrained_layout=True)
    ax1 = fig.add_subplot(111)

    df = pd.read_csv('2015StreetTreesCensus_TREES.csv')

    def onclick(event):
        global ix
        global iy
        ix, iy = event.xdata, event.ydata
        print(ix, iy)

    cid = fig.canvas.mpl_connect('button_press_event', onclick)
    #print(cid)
    plt.scatter(df['x_sp'],df['y_sp'],s=0.05,c='green')

    ratio = 1.0
    xMin, xMax = ax1.get_xlim()
    yMin, yMax = ax1.get_ylim()
    ax1.set_aspect( abs( ( xMax - xMin ) / ( yMin - yMax ) ) * ratio )

    plt.show()

else:
    ix,iy = 0,0

Lidar found for:  NY_NewYorkCity/
974100.9521937321 206554.0125270538
979864.4616359381 187114.23166126764
988341.8580730688 184738.56656391674


In [10]:
# S3 retrieve block

if ix != 0 and iy != 0:
    transformer = Transformer.from_crs("epsg:2263", "epsg:4326")
    lat, lon = transformer.transform(ix, iy)
    print(lat,lon)
else:
    pass
boxSize = 100

lidar_df = stackTiles(lat,lon,boxSize,prefix=lidarArea)

xmean = lidar_df['X'].mean()
ymean = lidar_df['Y'].mean()
zmean = lidar_df['Z'].mean()

singleReturns = lidar_df[lidar_df['number_of_returns'] - lidar_df['return_number'] == 0 ]
multipleReturns = lidar_df[lidar_df['number_of_returns'] - lidar_df['return_number'] > 0 ]

fig = plt.figure(figsize=(15,225), dpi=300, constrained_layout=True)
ax1 = fig.add_subplot(1, 1, 1, projection='3d') ############         
#ax1.scatter3D(lidar_df['X'], lidar_df['Y'], lidar_df['Z'], zdir='z', s=0.01, c=lidar_df['intens'], cmap='bone', marker='+', depthshade=True)
ax1.scatter3D(singleReturns['X'], singleReturns['Y'], singleReturns['Z'], zdir='z', s=0.01, c=singleReturns['intens'], cmap='bone', marker='o', depthshade=True)
ax1.scatter3D(multipleReturns['X'], multipleReturns['Y'], multipleReturns['Z'], zdir='z', s=2, c=multipleReturns['Z'], cmap='summer', marker='+', depthshade=True)
ax1.view_init(30, 225)         
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_zticks([])
ax1.grid(False)
ax1.set_axis_off()           
ax1.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax1.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax1.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax1.set_xlim3d(xmean-boxSize/2, xmean+boxSize/2)
ax1.set_ylim3d(ymean-boxSize/2, ymean+boxSize/2)
ax1.set_zlim3d(zmean-boxSize/2, zmean+boxSize/2)
plt.show()

40.673740456347005 -73.9852487204701


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/joe/opt/anaconda3/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/Users/joe/opt/anaconda3/lib/python3.9/tkinter/__init__.py", line 814, in callit
    func(*args)
  File "/Users/joe/opt/anaconda3/lib/python3.9/site-packages/matplotlib/backends/_backend_tk.py", line 252, in idle_draw
    self.draw()
  File "/Users/joe/opt/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_tkagg.py", line 9, in draw
    super().draw()
  File "/Users/joe/opt/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py", line 431, in draw
    self.renderer = self.get_renderer(cleared=True)
  File "/Users/joe/opt/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py", line 447, in get_renderer
    self.renderer = RendererAgg(w, h, self.figure.dpi)
  File "/Users/joe/opt/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_ag

In [11]:
#experimenting with automatically setting hyperparameters 
points = multipleReturns[['X','Y','Z']].to_numpy()
distances = pdist(points)
try:
    distances = np.reshape(distances,[len(points),-1])
except:
    distances = np.reshape(distances,[len(points)-1,-1])
avgMinDist = np.mean(np.amin(distances,axis=1))
avgDist = np.mean(distances)
stdDist = np.std(distances)
#print(avgDist,stdDist)  
xmin,xmax = multipleReturns['X'].min(),multipleReturns['X'].max()
ymin,ymax = multipleReturns['Y'].min(),multipleReturns['Y'].max()
zmin,zmax = multipleReturns['Z'].min(),multipleReturns['Z'].max()

eps = avgDist**0.33  #len(multipleReturns) / (0.025*(xmax-xmin)*(ymax-ymin)*(zmax-zmin))
clustering = DBSCAN(eps=eps,min_samples=20).fit(points)
clusters = clustering.labels_

multipleReturns['cluster'] = clusters

#print(multipleReturns.head())

###

fig = plt.figure(figsize=(15,15), dpi=300, constrained_layout=True)
ax1 = fig.add_subplot(1, 1, 1, projection='3d') ############         
ax1.scatter3D(singleReturns['X'], singleReturns['Y'], singleReturns['Z'], zdir='z', s=0.01, c='gainsboro', marker='o', depthshade=True)
ax1.scatter3D(multipleReturns['X'], multipleReturns['Y'], multipleReturns['Z'], zdir='z', s=2, c=multipleReturns['cluster'], cmap='tab20', marker='+', depthshade=True)
ax1.view_init(30, 225)        
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_zticks([])
ax1.grid(False)
ax1.set_axis_off()           
ax1.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax1.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax1.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax1.set_xlim3d(xmean-boxSize/2, xmean+boxSize/2)
ax1.set_ylim3d(ymean-boxSize/2, ymean+boxSize/2)
ax1.set_zlim3d(zmean-boxSize/2, zmean+boxSize/2)
plt.show()

/var/folders/5j/md25mrks3v1c7ghj1xzvpsb80000gn/T/ipykernel_6548/457682910.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multipleReturns['cluster'] = clusters


In [12]:
maxSearch = multipleReturns [multipleReturns['cluster'] != -1]
maxSearch['seeds'] = 1
xs = maxSearch['X'].to_numpy()
ys = maxSearch['Y'].to_numpy()
zs = maxSearch['Z'].to_numpy()
seeds = maxSearch['seeds'].to_numpy()

neighborhood = 4
changes = 1
while changes > 0:
    changes = 0
    i = 0
    for x,y,z,seed in zip(xs,ys,zs,seeds):
        if seed == 1:
            zsearch = zs[xs > x - neighborhood]
            xsearch = xs[xs > x - neighborhood]
            ysearch = ys[xs > x - neighborhood]
            
            zsearch = zsearch[xsearch < x + neighborhood]
            ysearch = ysearch[xsearch < x + neighborhood]
            
            zsearch = zsearch[ysearch > y - neighborhood]
            ysearch = ysearch[ysearch > y - neighborhood]
            
            zsearch = zsearch[ysearch < y + neighborhood]

            zmax = np.max(zsearch)
            if z < zmax:
                seeds[i] = 0
                changes += 1
            else:
                pass
        else:
            pass
        i+=1

maxSearch['seeds'] = seeds      
localMaxima = maxSearch[maxSearch['seeds']>0]

#

centers=localMaxima[['X','Y','Z']].to_numpy()
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(centers)
points = maxSearch[['X','Y','Z']].to_numpy()
distances, indices = nbrs.kneighbors(points)
maxSearch['nearestNeighbor'] = indices

#

fig = plt.figure(figsize=(15,15), dpi=300, constrained_layout=True)
ax1 = fig.add_subplot(1, 1, 1, projection='3d') ############         
ax1.scatter3D(singleReturns['X'], singleReturns['Y'], singleReturns['Z'], zdir='z', s=0.005, c='gainsboro', marker='o', depthshade=True)
ax1.scatter3D(maxSearch['X'], maxSearch['Y'], maxSearch['Z'], zdir='z', s=0.11, c=maxSearch['nearestNeighbor'], cmap='tab20', marker='o', depthshade=True)
ax1.scatter3D(localMaxima['X'], localMaxima['Y'], localMaxima['Z'], zdir='z', s=100, c='black', marker='+', depthshade=True)
ax1.view_init(30, 225)        
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_zticks([])
ax1.grid(False)
ax1.set_axis_off()           
ax1.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax1.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax1.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax1.set_xlim3d(xmean-boxSize/2, xmean+boxSize/2)
ax1.set_ylim3d(ymean-boxSize/2, ymean+boxSize/2)
ax1.set_zlim3d(zmean-boxSize/2, zmean+boxSize/2)
plt.show()

/var/folders/5j/md25mrks3v1c7ghj1xzvpsb80000gn/T/ipykernel_6548/236384428.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maxSearch['seeds'] = 1
/var/folders/5j/md25mrks3v1c7ghj1xzvpsb80000gn/T/ipykernel_6548/236384428.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maxSearch['seeds'] = seeds
/var/folders/5j/md25mrks3v1c7ghj1xzvpsb80000gn/T/ipykernel_6548/236384428.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [44]:
# calendar date time picker
pn.extension()
datetime_picker = pn.widgets.DatetimePicker(name='Datetime Picker', value=datetime.datetime(2022, 12, 8, 12, 00))
#datetime_picker.value
datetime_picker
#pn.Row(datetime_picker.controls(jslink=True), datetime_picker)

DatetimePicker(name='Datetime Picker', value=datetime.datetime(2022, ...)

In [46]:
# solar retrieve block

#date = datetime.datetime(year, month, day, hour+4, minute, 0, 0, tzinfo=datetime.timezone.utc)
date = datetime_picker.value
#date = date.replace(tzinfo=datetime.timezone.utc)

tz = tzwhere.tzwhere()
timezone_str = tz.tzNameAt(lat,lon) 
zone = pytz.timezone(timezone_str)
date = zone.localize(date)

az = get_azimuth(lat, lon, date)
amp = get_altitude(lat, lon, date)
rad = radiation.get_radiation_direct(date, amp)
print('Solar details: azimuth {}, amplitude {}, clear sky radiation {}'.format(az,amp,rad))

/Users/joe/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


Solar details: azimuth 179.0003349765064, amplitude 24.140650350058962, clear sky radiation 874.2790568077351


/Users/joe/opt/anaconda3/lib/python3.9/site-packages/pysolar/solartime.py:111: UserWarning: I don't know about leap seconds after 2021
  warnings.warn \


In [47]:
#species leaf area index in dense forest settting

lai = 0.75
laiDF = pd.read_csv('lai.csv')
laiDF = laiDF[laiDF['Country'] == 'USA']
speciesList = laiDF['Dominant_species'].unique()

for species in speciesList:
    tempLaiDF = laiDF[laiDF['Dominant_species'] == species]
    speciesLaiMean = tempLaiDF['Corrected_total_LAI_HSA'].mean()
    print('{} leaf area index: {} m2 m-2'.format(species,speciesLaiMean))   

Thuja plicata, Pseudotsuga menziesii, Tsuga heterophylla leaf area index: 6.892142857142857 m2 m-2
Pinus ponderosa leaf area index: 3.397906976744186 m2 m-2
Adenostoma fasciculatum leaf area index: 1.29 m2 m-2
Heath; deciduous(35-60%), evergreen(30-60%) leaf area index: 0.5299999999999999 m2 m-2
Shrub; Deciduous (70-100%), evergreen(0-30%) leaf area index: 1.3266666666666667 m2 m-2
Betula nana, Eriophorum vaginatum, Sphagnum mosses leaf area index: 2.53 m2 m-2
Sphagnum mosses, Epriophorum vaginatum, Betual nana leaf area index: 1.22 m2 m-2
Salix pulchra, Betual nana leaf area index: 3.47 m2 m-2
Fagus grandifolia, Acer saccharum, Betula alleghaniensis leaf area index: 7.0 m2 m-2
Quercus douglasii leaf area index: 0.83 m2 m-2
Iva frutescens leaf area index: 0.84 m2 m-2
Rhus copallina leaf area index: 1.52 m2 m-2
Elaeagnus umbellata leaf area index: 5.8 m2 m-2
Rhododendron maximum leaf area index: 8.36 m2 m-2
Morella cerifera leaf area index: 9.75 m2 m-2
Quercus rubra, Carya spp. leaf are

In [48]:
#converting solar azimuth and amplitude to a vector normal 

solarX = math.cos(amp) * math.cos(-az)  
solarY = math.cos(amp) * math.sin(-az) 
solarZ = -math.sin(amp) 
#print(solarX,solarY,solarZ)

# define a function to calculate the equation of a plane given a normal vector
def plane_equation(normal,x=1,y=1,z=1):
  a, b, c = normal
  d = -(a * x + b * y + c * z)
  return a, b, c, d

# define a function to project a 3D point onto a plane
def project_point(point, plane_equation):
  # unpack the point coordinates
  x, y, z = point
  a, b, c, d = plane_equation
  # calculate the distance from the point to the plane
  distance = abs(a * x + b * y + c * z + d) / math.sqrt(a**2 + b**2 + c**2)
  # calculate the coordinates of the projection of the point onto the plane
  x_proj = x - a * distance
  y_proj = y - b * distance
  z_proj = z - c * distance
  # return the coordinates of the projected point
  return x_proj, y_proj, z_proj

# define a function to calculate the area of a 2D shape
def area_2d(points):
  # calculate the area of the 2D shape
  # using the Shoelace formula
  x_sum = 0
  y_sum = 0
  for i in range(len(points)):
    x, y, z = points[i]
    x_next, y_next, z_next = points[(i + 1) % len(points)]
    x_sum += x * y_next
    y_sum += y * x_next
  area = 0.5 * abs(x_sum - y_sum)
  
  # return the area
  return area

# define a normal vector for the plane
normal = (solarX,solarY,solarZ)
# calculate the equation of the plane
plane_equation = plane_equation(normal)

clusters = maxSearch['nearestNeighbor'].unique()
#clusters = clusters[clusters>=0]

for cluster in clusters:
    temp = maxSearch[maxSearch['nearestNeighbor'] == cluster]
    points = temp[['X','Y','Z']].to_numpy()
    # project the 3D points onto the plane
    projected_points = [project_point(point, plane_equation) for point in points]
    # calculate the area of the 2D shape formed by the projected points
    area = area_2d(projected_points)*0.0929
    watts = area * rad * lai
    # print the area
    print('Cluster {}: \n{} square meters \n{} watts clear sky radiation blocked \n'.format(cluster,area,watts))

Cluster 105: 
1.927675 square meters 
1263.9944106238881 watts clear sky radiation blocked 

Cluster 4: 
11.7054 square meters 
7675.339553667945 watts clear sky radiation blocked 

Cluster 81: 
8.6397 square meters 
5665.131575326341 watts clear sky radiation blocked 

Cluster 93: 
1.13221875 square meters 
742.4063556375246 watts clear sky radiation blocked 

Cluster 108: 
8.151975 square meters 
5345.325760590177 watts clear sky radiation blocked 

Cluster 117: 
8.1752 square meters 
5360.554608910947 watts clear sky radiation blocked 

Cluster 0: 
0.058062499999999996 square meters 
38.072120801924335 watts clear sky radiation blocked 

Cluster 58: 
0.9289999999999999 square meters 
609.1539328307894 watts clear sky radiation blocked 

Cluster 1: 
6.3172 square meters 
4142.246743249368 watts clear sky radiation blocked 

Cluster 11: 
1.2599562499999999 square meters 
826.1650214017582 watts clear sky radiation blocked 

Cluster 116: 
11.84475 square meters 
7766.712643592565 watts